In [2]:
%load_ext autoreload
%autoreload 2
from tqdm.notebook import tqdm

In [59]:
# CLEAN & CLUSTER
from mods import cluster
cluster.run()

————————————————————————————————————————————————————————————
Loading & parsing listings data
————————————————————————————————————————————————————————————
  > 'listings.csv' loaded
  ——————————————————————————————
  > Searching listings data for alias host accounts...
    > Found 55/55 active alias host accounts...
  >>> Generated 44 new CorpGroup host IDs
  > Parsing license numbers
    > Imported + licenses parsed
  ——————————————————————————————
  > Importing ISD registry data...
    > Imported + licenses parsed
  ——————————————————————————————
  > Parsing license statuses...



    > Matched 747 to ISD data
  >>> 17% of total listings have an Active license
  ——————————————————————————————
  > Searching for license details of listings in ISD data...
  > Defining licenses used by multiple hosts 



————————————————————————————————————————————————————————————
CREATING CLUSTER LAYER under host_id ...
————————————————————————————————————————————————————————————
  > 1246 host_id index names found in cache
  > DBSCANning with 2 min_samples | epsilon = 300 ft.



Hosts: found 1246 in cache | 0 new
  > 0 outliers not clustered
  > 339 sub-classes generated & re-assigned
  ——————————————————————————————
  > 338 new centroids identified
  > 0 centroids recognized from cache
  ——————————————————————————————
  > Getting cluster params...



  > Found 338 centroid_groups
  ——————————————————————————————
  > Calculating centroid_group stats for 338 clusters


...


  > Found 69 sub_centroids
  ——————————————————————————————
  > Calculating sub_centroid stats for 69 clusters



  ——————————————————————————————
  > Analyzing sub_centroid radial stats...
  > Calculating listing distances from associated centroids...



  ——————————————————————————————
  > Calculating centroid location confidence...



Reverse geocoding cluster centroids with nominatim
  ——————————————————————————————
  > Found 5849 pre-geocoded locations in cache



  > 69 retrieved from the cache
  > 0 new locations geocoded & added to cache
  > Geocode completed
————————————————————————————————————————————————————————————
—————————————————————————————— 
Clustering completed
——————————————————————————————


In [49]:
# PREP DATA FOR MAPPING
from mods.map import *
sub_mode(False)
centroids, listings, sub_centroids = load_data()
centroids, listings = big_centroid_focus(centroids, listings)
centroids = assess_isd_matches(centroids, listings)
centroids = get_city_regions(centroids)
centroids = get_best_addresses(centroids)
sub_centroids = get_best_addresses(sub_centroids, True)
global_search(centroids, listings)

————————————————————————————————————————————————————————————
——————————————————————————————
> Loading ADCO STR building database... data_raw/STRbldgsGPSplus.csv
  > 223 STR buildings found
——————————————————————————————
> Centroid data loaded from clustered/centroids.csv
  > 199 hosts | 338 centroids | 1625 listings
> Sub centroid data loaded from clustered/sub_centroids.csv
  > 37 hosts | 69 centroids | 659 listings
> Listings data loaded from clustered/listings.csv
  > 1246 hosts | 338 centroids | 3339 listings
——————————————————————————————
> Calculating centroids map element radius (alternate)
> Calculating centroids map element opacity
> Applying license status color key
  ——————————————————————————————
  > Identifying big centroids...
    > Would drop 0 centroids (0 listings)
    > Would remain: 338 centroids (3339 listings)
———————————————
> Assessing ISD matches for centroids...
> Getting city regions for centroids...
> Getting best addresses for centroids...
> Getting best add

## make home page

In [50]:
# HOME PAGE
m = map_listings(listings)
page = make_home_page(m, listings)
Html_file= open('airbnb_tracker/home.html',"w")
Html_file.write(page)
Html_file.close()

## make high-conf centroids page

In [51]:
# TOP CENTROIDS PAGE
m = map_top_centroids(centroids, listings)
page = make_top_centroids_page(m, centroids, listings)
Html_file= open('airbnb_tracker/clusters.html',"w")
Html_file.write(page)
Html_file.close()

## make host focus pages

In [41]:
# HOST FOCUS PAGES
for host in tqdm(listings.host_id.unique()):
    m = map_focus_data('host_id', host, centroids, listings, sub_centroids)
    page = make_host_focus_page('host_id', host, m, centroids, listings, sub_centroids)
    Html_file= open(f'airbnb_tracker/host/{host}.html',"w")
    Html_file.write(page)
    Html_file.close()

## make centroid focus pages

In [57]:
# CENTROID FOCUS PAGES
for c_group in tqdm(listings.centroid_group.dropna().unique()):
    m = map_focus_data('centroid_group', c_group, centroids, listings, sub_centroids)
    page = make_centroid_page('centroid_group', c_group, m, listings, centroids, sub_centroids)
    Html_file= open(f'airbnb_tracker/cluster/{c_group}.html',"w")
    Html_file.write(page)
    Html_file.close()

## make sub-centroid focus pages

In [46]:
# SUB_CENTROID FOCUS PAGES

for sub_c_group in tqdm(listings.sub_centroid.dropna().unique()):
    m = map_one_sub(sub_c_group, listings, sub_centroids)
    page = make_sub_centroid_page(sub_c_group, m, listings, sub_centroids)
    Html_file= open(f'airbnb_tracker/sub_cluster/{sub_c_group}.html',"w")
    Html_file.write(page)
    Html_file.close()